In [50]:
import os
import pandas as pd

def calculate_monthly_daylight_averages(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.csv'):
            input_path = os.path.join(input_folder, filename)
            print(f"Processing: {input_path}")
            
            try:
                df = pd.read_csv(input_path, skiprows=2)
                df.columns = df.columns.str.strip()

                # Filter out nighttime hours (where GHI is 0)
                daylight_df = df[df['GHI'] > 0]
                
                # If no daylight hours, skip this file or handle differently
                if daylight_df.empty:
                    print(f"No daylight hours found in {filename}, skipping...")
                    continue

                # Drop unnecessary columns
                daylight_df = daylight_df.drop(columns=['Day', 'Hour', 'Minute'], errors='ignore')

                # Compute monthly averages for daylight hours only
                monthly_avg = daylight_df.groupby(['Year', 'Month']).mean(numeric_only=True).reset_index()

                # Create a single datetime column from Year and Month
                monthly_avg['Date'] = pd.to_datetime(monthly_avg[['Year', 'Month']].assign(DAY=1))

                # Reorder columns so 'Date' is first
                cols = ['Date'] + [col for col in monthly_avg.columns if col not in ['Year', 'Month', 'Date']]
                monthly_avg = monthly_avg[cols]

                # Output file path
                output_filename = f"monthly_daylight_avg_{filename}"
                output_path = os.path.join(output_folder, output_filename)

                # Save to CSV
                monthly_avg.to_csv(output_path, index=False)
                print(f"Saved daylight averages to: {output_path}\n")

            except Exception as e:
                print(f"Failed to process {filename}: {e}")

# Usage
input_dir = r'C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Solar_Irradiance'
output_dir = r'C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Cleaned_Solar_Irradiance'

calculate_monthly_daylight_averages(input_dir, output_dir)

Processing: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Solar_Irradiance\623099_40.01_-99.74_1998.csv
Saved daylight averages to: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Cleaned_Solar_Irradiance\monthly_daylight_avg_623099_40.01_-99.74_1998.csv

Processing: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Solar_Irradiance\623099_40.01_-99.74_1999.csv
Saved daylight averages to: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Cleaned_Solar_Irradiance\monthly_daylight_avg_623099_40.01_-99.74_1999.csv

Processing: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Solar_Irradiance\623099_40.01_-99.74_2000.csv
Saved daylight averages to: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Cleaned_Solar_Irradiance\monthly_daylight_avg_623099_40.01_-99.74_2000.csv

Processing: C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\Solar_Irradiance\623099_40.01_-99.74_2001.csv
Saved daylight averages to: C:\Users\bradl\Desktop\BigData-Project-Rewo

In [54]:
import os
import pandas as pd

def merge_monthly_averages(input_folder, output_file):
    """
    Merges all monthly average CSV files in a folder into a single consolidated file.
    
    Args:
        input_folder (str): Path to folder containing monthly average CSV files
        output_file (str): Path for the output consolidated CSV file
    """
    # Initialize an empty list to store DataFrames
    all_data = []
    
    # Get list of files to process
    files_to_process = [f for f in os.listdir(input_folder) 
                       if f.lower().endswith('.csv') and f.startswith('monthly_')]
    
    if not files_to_process:
        print("No monthly average files found in the input folder.")
        return
    
    print(f"Found {len(files_to_process)} monthly average files to merge...")
    
    for filename in files_to_process:
        file_path = os.path.join(input_folder, filename)
        try:
            # Read each file
            df = pd.read_csv(file_path)
            
            # Add a column to track the source file (optional)
            df['Source_File'] = filename
            
            # Append to our list
            all_data.append(df)
            print(f"Processed: {filename}")
            
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    
    if not all_data:
        print("No valid data to merge.")
        return
    
    # Concatenate all DataFrames
    merged_df = pd.concat(all_data, ignore_index=True)
    
    # Sort by Date (if available) or Year/Month
    if 'Date' in merged_df.columns:
        merged_df['Date'] = pd.to_datetime(merged_df['Date'])
        merged_df = merged_df.sort_values('Date')
    elif 'Year' in merged_df.columns and 'Month' in merged_df.columns:
        merged_df = merged_df.sort_values(['Year', 'Month'])
    
    # Save the merged file
    merged_df.to_csv(output_file, index=False)
    print(f"\nSuccessfully merged {len(all_data)} files into {output_file}")
    print(f"Total records: {len(merged_df)}")

# Example usage:
input_dir = r'C:\Users\bradl\Desktop\BigData-Project-Rework\Raw-Data\cleaned_Solar_Irradiance'
output_file = r'C:\Users\bradl\Desktop\BigData-Project-Rework\WebDashboard\data\consolidated_solar_irradiance.csv'

merge_monthly_averages(input_dir, output_file)

Found 26 monthly average files to merge...
Processed: monthly_daylight_avg_623099_40.01_-99.74_1998.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_1999.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2000.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2001.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2002.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2003.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2004.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2005.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2006.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2007.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2008.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2009.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2010.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2011.csv
Processed: monthly_daylight_avg_623099_40.01_-99.74_2012.csv
Processed: monthly_daylight_avg_623099_40.